# This notebook prepares data for modeling

## 1. Load packages & helper functions

In [17]:
import os
import pandas as pd
import numpy as np
import math

def get_payroll_file_name(year, month):
    file_name = f"{month_names[month]}{year}"
    file_path = f"../data/{file_name}.xlsx"
    return file_path


def load_file_excel(file_path, sheet_name=0):
    if not os.path.exists(file_path):
        return None
    
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    return df

month_names = {
    1:"Jan",
    2:"Feb",
    3:"March",
    4:"April",
    5:"May",
    6:"June",
    7:"July",
    8:"Aug",
    9:"Sept",
    10:"Oct",
    11:"Nov",
    12:"Dec",
}

## 2. Clean & merge payroll data

In [141]:
 # Keep only needed columns
def clean_payroll_data(df, year, month):
    df = df[["EECode", "EarnHours", "EarnRate",	"EarnAmount","Dist Position Desc"	]]
    df.is_copy = None
    df["Year"] = year
    df["Month"] = month
    return df


def merge_payroll_data():
    result = None
    for year in [2020, 2021, 2022]:
        for month in range(1, 13):
            name = get_payroll_file_name(year, month)
            df = load_file_excel(name)
            if df is not None:
                df = clean_payroll_data(df, year, month)
                if result is None:
                    result = df
                else:
                    result = pd.concat([result, df])
    return result

merged = merge_payroll_data()
merged.to_csv("../data/merged_payroll.csv", index=False)
merged

C:\Users\weilin\AppData\Local\Temp\ipykernel_30032\4234863202.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"] = year
C:\Users\weilin\AppData\Local\Temp\ipykernel_30032\4234863202.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Month"] = month
C:\Users\weilin\AppData\Local\Temp\ipykernel_30032\4234863202.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,EECode,EarnHours,EarnRate,EarnAmount,Dist Position Desc,Year,Month
0,938,12.19,8.56,104.35,WILD Steward,2020,12
1,938,14.69,8.80,129.28,WILD Steward,2020,12
2,939,32.73,8.56,280.17,WILD Steward,2020,12
3,939,24.00,8.80,211.20,WILD Steward,2020,12
4,1160,33.90,8.56,290.18,WILD Steward,2020,12
...,...,...,...,...,...,...,...
15,1483,55.83,13.00,725.79,WILD Steward,2022,12
16,1484,41.18,13.00,535.34,WILD Steward,2022,12
17,1423,98.43,23.89,2351.52,Adult Education Specialist,2022,12
18,1485,64.63,13.00,840.19,WILD Steward,2022,12


## 3. Clean & merge applicant data

In [52]:
df_list = load_file_excel(f"../data/WILD Applicant Data (2016-2022).xlsx", sheet_name=[-1,-2,-3])
applicant_2022 = df_list[-1]
applicant_2021 = df_list[-2]
applicant_2020 = df_list[-3]

applicant_2022 = applicant_2022.rename(columns = {
'Date of Birth (MM/DD/YYYY)':'Date of Birth',
'Current School (2021-2022)':'School',
'School for the 2022-2023 school year': 'Next Year School',
applicant_2022.columns[6]:'Zipcode',
applicant_2022.columns[7]:'Program',
applicant_2022.columns[8]:'First Paying Job',
applicant_2022.columns[9]:'Know anyone who has been employeed',
applicant_2022.columns[10]:'Applied Before',
applicant_2022.columns[11]:'Speaking in front of a crowd rate',
applicant_2022.columns[12]:'Unconfortable to work with',
applicant_2022.columns[13]:'Extra activities',
applicant_2022.columns[14]:'Commit Summer',
applicant_2022.columns[16]: 'Commit Weekday',
applicant_2022.columns[18]:'Hear about source'
})

applicant_2022 = applicant_2022[['ID Number', 'Preferred Pronouns', 'Date of Birth', 'School',
       'Current Grade Level', 'Next Year School', 'Zipcode', 'Program',
       'First Paying Job', 'Know anyone who has been employeed',
       'Applied Before', 'Speaking in front of a crowd rate',
       'Unconfortable to work with', 'Extra activities', 'Commit Summer','Commit Weekday',
       'Hear about source']]

applicant_2022['Application Year'] = 2022


applicant_2021 = applicant_2021.rename(columns = {
'What is your preferred gender pronoun?':'Preferred Pronouns',
'Date of Birth (ie: 01/01/1999)':'Date of Birth',
'Current Grade level (i.e. Freshman, Sophomore, Junior, Senior)':'Current Grade Level',
'Current School (2021-2022)':'School',
'How did you hear about us?':'Hear about source',
applicant_2021.columns[6]: 'Hear about source other',
'School for the 2022-2023 school year': 'Next Year School',
'What Zip Code do you live in?':'Zipcode',
applicant_2021.columns[10]:'Program',
applicant_2021.columns[11]:'First Paying Job',
applicant_2021.columns[13]:'Know anyone who has been employeed',
applicant_2021.columns[14]:'Applied Before',
applicant_2021.columns[16]:'Unconfortable to work with',
applicant_2021.columns[17]:'Commit Summer',
applicant_2021.columns[19]:'Commit Weekday',
applicant_2021.columns[21]:'Speaking in front of a crowd rate',

})

# get from hear about source other
applicant_2021['Hear about source'] = np.where(applicant_2021['Hear about source'] == 'Other', applicant_2021['Hear about source other'], applicant_2021['Hear about source'])

applicant_2021 = applicant_2021[['ID Number', 'Preferred Pronouns', 'Date of Birth','Current Age', 'School',
       'Current Grade Level',  'Zipcode', 'Program',
       'First Paying Job', 'Know anyone who has been employeed',
       'Applied Before', 'Speaking in front of a crowd rate',
       'Unconfortable to work with',  'Commit Summer',
    'Commit Weekday',
       'Hear about source']]

applicant_2021['Next Year School'] = None
applicant_2021['Extra activities'] = None
applicant_2021['Application Year'] = 2021



applicant_2020 = applicant_2020.rename(columns = {
    'What is your preferred gender pronoun?':'Preferred Pronouns',
    applicant_2020.columns[2]:'Preferred Pronouns Alter 1',
    applicant_2020.columns[3]:'Preferred Pronouns Alter 2',
    'Date of Birth (ie: 01/01/1999)':'Date of Birth',
    'How did you hear about us?':'Hear about source',
    applicant_2020.columns[9]: 'Hear about source other',
    applicant_2020.columns[10]: 'Hear about source other_1',
    applicant_2020.columns[11]: 'Hear about source other_2',
    applicant_2020.columns[12]: 'Hear about source other_3',
    applicant_2020.columns[13]: 'Hear about source other_4',
    applicant_2020.columns[14]: 'Hear about source other_5',
    applicant_2020.columns[15]: 'Hear about source other_6',
    applicant_2020.columns[16]: 'Current Grade Level',
    applicant_2020.columns[17]: 'Zipcode',
    applicant_2020.columns[20]:  'Program',
    applicant_2020.columns[21]: 'First Paying Job',
    applicant_2020.columns[24]:  'Know anyone who has been employeed',
    applicant_2020.columns[26]: 'Applied Before',
    applicant_2020.columns[29]: 'Unconfortable to work with',
    applicant_2020.columns[30]: 'Unconfortable to work with 1',
    applicant_2020.columns[31]: 'Unconfortable to work with 2',
    applicant_2020.columns[32]: 'Unconfortable to work with 3',
    applicant_2020.columns[33]: 'Unconfortable to work with 4',
    applicant_2020.columns[34]:'Commit Summer',
    applicant_2020.columns[38]: 'Commit Weekday',
    applicant_2020.columns[42]: 'Speaking in front of a crowd rate',
})


# todo, get the data from the alter sources
applicant_2020['Preferred Pronouns'] = np.where(pd.isnull(applicant_2020['Preferred Pronouns']), applicant_2020['Preferred Pronouns Alter 1'], applicant_2020['Preferred Pronouns'])
applicant_2020['Preferred Pronouns'] = np.where(pd.isnull(applicant_2020['Preferred Pronouns']), applicant_2020['Preferred Pronouns Alter 2'], applicant_2020['Preferred Pronouns'])


applicant_2020['Hear about source'] = np.where(pd.isnull(applicant_2020['Hear about source']), applicant_2020['Hear about source other'], applicant_2020['Hear about source'])
applicant_2020['Hear about source'] = np.where(pd.isnull(applicant_2020['Hear about source']), applicant_2020['Hear about source other_1'], applicant_2020['Hear about source'])
applicant_2020['Hear about source'] = np.where(pd.isnull(applicant_2020['Hear about source']), applicant_2020['Hear about source other_2'], applicant_2020['Hear about source'])
applicant_2020['Hear about source'] = np.where(pd.isnull(applicant_2020['Hear about source']), applicant_2020['Hear about source other_3'], applicant_2020['Hear about source'])
applicant_2020['Hear about source'] = np.where(pd.isnull(applicant_2020['Hear about source']), applicant_2020['Hear about source other_4'], applicant_2020['Hear about source'])
applicant_2020['Hear about source'] = np.where(pd.isnull(applicant_2020['Hear about source']), applicant_2020['Hear about source other_5'], applicant_2020['Hear about source'])
applicant_2020['Hear about source'] = np.where(pd.isnull(applicant_2020['Hear about source']), applicant_2020['Hear about source other_6'], applicant_2020['Hear about source'])

applicant_2020['Unconfortable to work with'] = np.where(pd.isnull(applicant_2020['Unconfortable to work with']), applicant_2020['Unconfortable to work with 1'], applicant_2020['Unconfortable to work with'])
applicant_2020['Unconfortable to work with'] = np.where(pd.isnull(applicant_2020['Unconfortable to work with']), applicant_2020['Unconfortable to work with 2'], applicant_2020['Unconfortable to work with'])
applicant_2020['Unconfortable to work with'] = np.where(pd.isnull(applicant_2020['Unconfortable to work with']), applicant_2020['Unconfortable to work with 3'], applicant_2020['Unconfortable to work with'])
applicant_2020['Unconfortable to work with'] = np.where(pd.isnull(applicant_2020['Unconfortable to work with']), applicant_2020['Unconfortable to work with 4'], applicant_2020['Unconfortable to work with'])


applicant_2020 = applicant_2020[['ID Number', 'Preferred Pronouns', 'Date of Birth','Current Age', 'School',
       'Current Grade Level',  'Zipcode', 'Program',
       'First Paying Job', 'Know anyone who has been employeed',
       'Applied Before', 'Speaking in front of a crowd rate',
       'Unconfortable to work with',  'Commit Summer',
    'Commit Weekday',
       'Hear about source']]

applicant_2020['Next Year School'] = None
applicant_2020['Extra activities'] = None
applicant_2020['Application Year'] = 2020

def is_float(v):
       try:
              f = float(v)
              return True
       except Exception as ex:
              print(ex)
              return False

# merge files
applicant_data = pd.concat([applicant_2022, applicant_2021,applicant_2020])

# drop erroneous row
applicant_data = applicant_data[applicant_data['ID Number'] != '\xa0'].reset_index(drop = True)

# flag rejected candidates
applicant_data['Rejected'] = applicant_data['ID Number'].apply(lambda x: math.isnan(x))

# convert existing IDs to int
applicant_data.loc[applicant_data['Rejected'] == False, 'ID Number'] = applicant_data.loc[applicant_data['Rejected'] == False, 'ID Number'].astype(int)

#save
applicant_data.to_csv("../data/applicants.csv", index=False)
applicant_data


,ID Number,Preferred Pronouns,Date of Birth,School,Current Grade Level,Next Year School,Zipcode,Program,First Paying Job,Know anyone who has been employeed,Applied Before,Speaking in front of a crowd rate,Unconfortable to work with,Extra activities,Commit Summer,Commit Weekday,Hear about source,Application Year,Current Age,Rejected
0,1489,He/Him/His (masculine pronouns),2006-12-13 00:00:00,First Coast High School,9th,First Coast High School,32218,Zoocamp,Yes,NaN,NaN,Very comfortable! I can confidently speak to a...,"Invertebrates (roaches, spiders, millipedes)",Nothing at this time,Yes,Yes,Jacksonville Zoo & Gardens website,2022,NaN,False
1,NaN,He/Him/His (masculine pronouns),2004-11-15 00:00:00,Jean Ribault High School,11th,Jean Ribault High School,NaN,No,No,NaN,No,"I can do it. I mean, I would rather speak to s...","Invertebrates (roaches, spiders, millipedes)",NaN,Not Sure,Yes,Jamie Lankenau,2022,NaN,True
2,1488,He/Him/His (masculine pronouns),2004-11-15 00:00:00,Jean Ribault High School,11th,Jean Ribault High School,NaN,No,No,NaN,NaN,"I can do it. I mean, I would rather speak to s...","Invertebrates (roaches, spiders, millipedes)","I run track at Jean Ribault High School, I als...",Yes,Yes,Friend,2022,NaN,False
3,NaN,He/Him/His (masculine pronouns),2006-09-25 00:00:00,Sandalwood senior high school,9,Sandalwood senior high school,32206,No,No,NaN,No,"I can do it. I mean, I would rather speak to s...",I am open to working with any of these animals,"Yes, Football wrestling and lacrosse",Yes,Yes,Vendor,2022,NaN,True
4,NaN,He/Him/His (masculine pronouns),2006-06-22 00:00:00,Andrew Jackson senior high school,10th,Andrew Jackson senior high school,32206,No,No,NaN,NaN,"I can do it, but I don't like it. It's tough f...",I am open to working with any of these animals,Football and soccer,Not Sure,Not Sure,Park vender,2022,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,NaN,She/Her/Hers,2003-08-13 00:00:00,William m raines,11 junior,None,32206,NaN,NaN,Yes,NaN,NaN,"Reptiles (snakes, lizards, turtles)",None,Yes,Yes,School Career Fair,2020,16.0,True
314,NaN,He/Him/His,2004-08-07 00:00:00,Edward H. White Senior High School,Sophomore,None,32218,NaN,Yes,NaN,NaN,Very comfortable! I can confidently speak to a...,I am open to working with any of these animals.,None,Yes,Yes,Jacksonville Zoo and Gardens Website,2020,15.0,True
315,NaN,She/Her/Hers,2004-08-05 00:00:00,Andrew Jackson High School,Sophmore,None,32218,NaN,Yes,NaN,NaN,Very comfortable! I can confidently speak to a...,"Invertebrates (roaches, spiders, millipedes)",None,NaN,Yes,School Career Fair,2020,15.0,True
316,NaN,She/Her/Hers,2004-07-11 00:00:00,Andrew Jackson High School,sophomore,None,32218,NaN,Yes,Yes,NaN,NaN,"Invertebrates (roaches, spiders, millipedes)",None,Yes,Yes,School Career Fair,2020,15.0,True
